In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from loguru import logger
from datetime import datetime
import glob
from matplotlib import pyplot as plt
from collections import Counter
import time
from fake_useragent import UserAgent
from collections import Counter
import urllib

from urllib.request import Request, urlopen

nest_asyncio.apply()


In [ ]:
print(f'{asyncio.get_event_loop()}')
print(f'{asyncio.get_event_loop_policy()}')

In [15]:
base_url = 'www.bookyogaretreats.com'

url='https://www.bookyogaretreats.com/yoga-weeks/6-day-urban-yoga-retreat-in-barcelona-catalonia'
# url='https://www.bookyogaretreats.com/?page=504'
print(url)

# r = requests.get(url)
# response = r.text.encode('utf-8')

r = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
response = urlopen(r).read()

soup = BeautifulSoup(response, 'html.parser')
title=soup.find('h1', {'class': 'listing-title__title title listing-title-new'}).text
soup.find('div', {'class': 'listing-organizer-card__content'}).text

https://www.bookyogaretreats.com/yoga-weeks/6-day-urban-yoga-retreat-in-barcelona-catalonia


'\nRetreat organizer: Yoga Weeks\n\nYoga Weeks is a modern yoga lifestyle team. Offering yoga retreats in Barcelona, Andorra, and Ibiza, as well as the Yoga Boat.\n\n\n\n\nBarcelona, Spain\n\n\nThe organizer also speaks: French, German, Spanish\n\n\n'

In [ ]:
retreat_links = []

pages_start=350
# max 504
pages_end = 351
# [pages_start, pages_end)

workers = 1

pages_number=int(pages_end-pages_start)

print(f'pages num {pages_number}')

pages_step = int(pages_number / workers)
diff = pages_number - pages_step * workers

workers_pages_start = [pages_step * i + pages_start for i in range(workers)]
workers_pages_end = [pages_step * i+pages_start-1 for i in range(1, workers + 1)]
workers_pages_end[-1] += diff

async def get_links(start, end):
    global retreat_links
    for page in range(start, end + 1):
        print(page)
        url = f'https://www.bookyogaretreats.com/?page={page}'

        async with aiohttp.ClientSession() as session:
            async with session.get(url,headers={'user-agent': 'Mozilla/5.0'}) as response:
                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                links = soup.findAll('a', {'class': 'js-showcard-link'}, href=True)
                for link in links:
                    retreat_link = 'https://www.bookyogaretreats.com' + link['href']
                    retreat_links.append(retreat_link)
                np.save(f'bookyogaretreats_links_{pages_start}_{pages_end}.npy', retreat_links)

        await session.close()


result = asyncio.run(asyncio.gather(*[get_links(workers_pages_start[i], workers_pages_end[i]) for i in range(workers)]))

In [ ]:
pages_start=351
pages_end=505
arr=np.load(f'bookyogaretreats_links_{pages_start}_{pages_end}.npy')
arr.shape

In [ ]:
a1= np.load(f'bookyogaretreats_links_1_200.npy')
a2 = np.load(f'bookyogaretreats_links_200_201.npy')
a3 = np.load(f'bookyogaretreats_links_201_350.npy')
a4 = np.load(f'bookyogaretreats_links_350_351.npy')
a5 = np.load(f'bookyogaretreats_links_351_505.npy')

links=np.concatenate([a1,a2,a3,a4,a5])

In [ ]:
np.save('bookyogaretreats_links.npy',links)

In [4]:
links=np.load('bookyogaretreats_links.npy')
links.shape

(5730,)

In [ ]:
lnk=np.unique(links)
np.save('bookyogaretreats_links.npy',lnk)

In [ ]:
retreats = pd.DataFrame(
    {'title': [0], 'duration': [0], 'center_title': [0], 'center_url': [0], 'location': [0], 'country': [0],
     'category': [0], 'price': [0],
     'deposit': [0], 'retreat_url': [0]})
retreats.to_csv('retreats_bookyogaretreats.csv', encoding='utf-8', index=False)

In [ ]:
retreats  = pd.read_csv('retreats_bookyogaretreats.csv')
retreats.head(5)

In [ ]:
retreats.info()

In [ ]:
links=np.load('bookyogaretreats_links.npy')

base_url = 'https://bookretreats.com'

pages_start=1
# max 5730
pages_end = 5731
# [pages_start, pages_end)

workers = 20

pages_number=int(pages_end-pages_start)

print(f'pages num {pages_number}')

pages_step = int(pages_number / workers)
diff = pages_number - pages_step * workers

workers_pages_start = [pages_step * i + pages_start for i in range(workers)]
workers_pages_end = [pages_step * i+pages_start-1 for i in range(1, workers + 1)]



async def get_retreat(start, end):
    global retreats
    # global img_links
    global links
    for i in range(start, end + 1):
        print(i)
        retreat_url = links[i]

        async with aiohttp.ClientSession() as retreat_session:
            async with retreat_session.get(retreat_url) as retreat_response:

                retreat_response = await retreat_response.text()
                soup = BeautifulSoup(retreat_response, 'html.parser')


                title = soup.findAll('div', {'class': 'col-span-4 sm:col-span-3 flex items-center'})[0].text


                duration =soup.findAll('div', {'class': 'mb-4 text-base text-gray-500 dark:text-gray-300'})[0].text.split(' ')[0]


                loc=soup.findAll('a', {'class': 'block text-sm leading-7 text-gray-500 dark:text-gray-400'})
                country=loc[-2].text
                location=country+' '+loc[-1].text
                category=loc[1].text

                deposit = soup.findAll('div', {'class': 'tabular-nums text-xl text-gray-500 dark:text-gray-200 font-semibold'})[
                              0].text[1:].replace(',', '')[3:]

                price = soup.findAll('div', {'class': 'tabular-nums text-2xl text-gray-500 dark:text-gray-200 font-extrabold'})[
                            0].text[1:].replace(',', '')[3:]

                center = soup.find('a', {'class': 'block text-gray-900 dark:text-gray-200 text-lg font-bold mb-2'})
                center_title = center.text
                center_half_url = center['href']
                center_url = base_url + center_half_url

                retreats  = retreats.append(
                    {'title': title, 'duration': duration, 'center_title': center_title, 'center_url': center_url,
                     'location': location, 'country': country, 'category': category,
                     'price': price, 'deposit': deposit, 'retreat_url': retreat_url}, ignore_index=True)
                retreats.to_csv('retreats_bookretreats.csv', encoding='utf-8', index=False)


result = asyncio.run(asyncio.gather(*[get_retreat(workers_pages_start[i], workers_pages_end[i]) for i in range(workers)]))